In [1]:
import pandas as pd
import USA_oil_data as oil
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set()

# monthly SD

In [ ]:
weekly_stocks = oil.weekly_stocks().to_frame() / 7
weekly_stocks['change'] = weekly_stocks['stocks'].diff(-1)
weekly_stocks['SD_change'] = weekly_stocks['change'].diff(-52)

weekly_stocks['year'] = weekly_stocks.index.year
weekly_stocks['month'] = weekly_stocks.index.month
week_info = weekly_stocks.index.isocalendar()
week_number = week_info['week']
weekly_stocks = pd.merge(week_number, weekly_stocks, left_index=True, right_index=True)

weekly_stocks = weekly_stocks[weekly_stocks['week'] <= 52]
weekly_stocks = weekly_stocks[weekly_stocks['year'] >= 2018]

grouped = weekly_stocks.groupby('year')

In [ ]:
pos = 0
width = 0.15
fig, ax = plt.subplots(figsize=(15,8))
for name, group in grouped:
    position = group['week']
    data = group['SD_change']
    data = data[:52]
    ax.bar(position + pos * width, data, width=width, label=name)
    pos += 1
plt.legend()
plt.xlabel('Week')
plt.ylabel('MBBL/D')
plt.title('weekly SD change (YOY)')
plt.show()

In [ ]:
monthly_SD_data = []
for name, group in grouped:
    year = name
    monthly_group = group.groupby('month')
    for n, g in monthly_group:
        week = n
        data = g['SD_change'].sum()
        monthly_SD_data.append([year, week, data])

monthly_SD_df = pd.DataFrame(monthly_SD_data, columns=['year', 'month', 'SD_change'])

grouped = monthly_SD_df.groupby('year')

In [ ]:
os = 0
width = 0.15
fig, ax = plt.subplots(figsize=(15,8))
for name, group in grouped:
    position = group['month']
    data = group['SD_change'] / 7
    ax.bar(position + pos * width, data, width=width, label=name)
    pos += 1

plt.legend()
plt.xlabel('month')
plt.ylabel('MBBL/D')
plt.title('monthly SD change (YOY)')
plt.show()

In [ ]:
monthly_SD_df['date'] = pd.to_datetime(monthly_SD_df['year'].astype(str) + '-' + monthly_SD_df['month'].astype(str) + '-1')

In [ ]:
monthly_SD_df.set_index('date', inplace=True)

In [ ]:
display(monthly_SD_df)

In [ ]:
#net imports
imports, exports = oil.imports_exports(only_crude=False)
imports_exports = pd.merge(imports, exports, on='period')
imports_exports['year'] = imports_exports.index.year
imports_exports = imports_exports[imports_exports['year'] > 2005]
net_imports = imports_exports['imports'] - imports_exports['exports']
net_imports.rename("net_imports", inplace=True)
net_imports = net_imports.groupby(pd.Grouper(freq='MS')).mean()

#production
production = oil.mbbl_production(daily=True)

#product supplied (EIA uses product supplied to represent U.S. petroleum consumption)
product_supplied = oil.monthly_product_supplied()
product_supplied = product_supplied * -1

#refinery/blender net inputs
net_refinery = oil.refinery_net_input(daily=True)

supply_demand = pd.merge(production, product_supplied, on='period')
supply_demand = pd.merge(supply_demand, net_imports, on='period')
#supply_demand = pd.merge(supply_demand, net_refinery, on='period')

changes = supply_demand.diff(1)
changes = supply_demand.diff(12)

sd = monthly_SD_df['SD_change'].rename('SD')



In [ ]:
changes = changes[changes.index > '2017-12-01']
fig, ax = plt.subplots(figsize=(10,5))
changes.plot(kind='bar', stacked=True, ax=ax)
plt.title('Change in Supply and Demand (YoY)')
ax.set_xlabel('Month')
ax.set_ylabel('Change (MBBL/D)')
plt.xticks(rotation=85)
plt.show()

In [ ]:
sd_data = sd[sd.index > '2017-12-01' ]
ax = sd_data.plot.bar(ylabel="MBBL", xlabel="Date", figsize=(15,8),
                 color='black', position=0, width=0.3)

changes.plot.bar(stacked=True, sharex=True, ax=ax, position=1, width=0.3)


plt.title('SD by Components')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=5)
plt.ylabel('YoY Change (MBBL/D)')
plt.xlabel('date')
plt.show()

# weekly SD
SD balance = (weekly change - last year's weekly change) / 7
Factors = change in supply - change in consumption - change in net imports
positive SD balance = increasing in stocks
negative SD balance = decreasing in stocks

Factors:
-
- U.S. Refiner Net Input of Crude Oil (Thousand Barrels per Day) [x] (refinery_inputs)
- U.S. Net Imports of Crude Oil (Thousand Barrels per Day) [x] (net_imports)
- U.S. Field Production of Crude Oil (Thousand Barrels per Day) [x] (production)
- U.S. Product Supplied of Crude Oil and Petroleum Products (Thousand Barrels per Day) [x] (product_supplied)

SD:
- U.S. Ending Stocks of Crude Oil (Thousand Barrels) / 7 [x] (stocks)


In [2]:
stocks = oil.weekly_stocks().to_frame() / 7
stocks['change'] = stocks['stocks'].diff(-1)
stocks['Stock_change'] = stocks['change'].diff(-52)
stocks = stocks.sort_index()

In [3]:
product_supplied = oil.weekly_product_supplied() * -1

/Users/nickgault/PycharmProjects/Kayak/Oil_analysis/USA_oil_data.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('period').sum()['value']


In [4]:
imports, exports = oil.imports_exports(only_crude=True)
net_imports = imports - exports
net_imports = net_imports.rename("net_imports")

In [5]:
refinery_inputs = oil.weekly_refinery_inputs()

/Users/nickgault/PycharmProjects/Kayak/Oil_analysis/USA_oil_data.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('period').sum()['value']


In [6]:
production = oil.weekly_field_production().sort_index()

In [7]:
master_df = pd.merge(product_supplied, net_imports, on='period')
#master_df = pd.merge(master_df, refinery_inputs, on='period')
master_df = pd.merge(master_df, production, on='period')

In [8]:
#change = master_df.diff(1)
#changes = change.diff(52)
changes = master_df.diff(52)

In [ ]:
sd_data = stocks['Stock_change'].rename('Storage Injections')

In [ ]:
start_date = '2022-01-01'
end_date = '2023-06-06'

changes = changes[start_date:end_date]
sd_data = sd_data[start_date:end_date]

In [ ]:
changes.rename(columns={'weekly_product_supplied': 'Consumption', 'net_imports': 'Net Imports', 'weekly_field_production': 'Production'}, inplace=True)

In [ ]:
sd_data.index = pd.to_datetime(sd_data.index)

ax = sd_data.plot.bar(ylabel="MBBL", xlabel="Date", figsize=(15,8),
                 color='black', position=0, width=0.3)

changes.plot.bar(stacked=True, sharex=True, ax=ax, position=1, width=0.3)

ax.xaxis.set_major_formatter(plt.FixedFormatter(sd_data.index.strftime('%Y-%m-%d')))

plt.title('change in Storage Injections by Components (YOY)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=5)
plt.ylabel('YoY Change (MBBL/D)')
plt.xlabel('date')
plt.show()

In [ ]:
sd_data = sd_data.to_frame()
sd_data['year'] = sd_data.index.year

week_info = sd_data.index.isocalendar()
week_number = week_info['week']

sd_data = pd.merge(week_number, sd_data, left_index=True, right_index=True)

In [ ]:
grouped = sd_data.groupby('year')

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
i = 0
week_sum = {i:0 for i in range(1, 53)}
week_count = {i:0 for i in range(1, 53)}
for name, group in grouped:
    group = group[group['week'] < 53]
    weeks = group['week']
    data = group['SD']
    for w, d in zip(weeks, data):
        week_sum[w] += d
        week_count[w] += 1

    ax.bar(weeks + i * width, data, width=width, label=name)
    i += 1

means = []
for s, count in zip(week_sum.values(), week_count.values()):
    means.append(s/count)

means_df = pd.DataFrame(means)
plt.plot(means_df.index+1, means_df, 'ro')

plt.xticks(range(1, 53))
ax.set_ylabel('Stocks')
ax.set_xlabel('Week')
#move legend to bottom
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=5)
plt.title('Weekly Change in Crude Stocks')

plt.show()


In [ ]:
print(product_supplied[-1])
print(production[-1])
print(net_imports[-1])

In [ ]:
i = 3
print(production[-i] + net_imports[-i] - product_supplied[-i])
print((stocks[i-1] - stocks[i]))

In [ ]:
display(stocks[0] - stocks[1])

In [ ]:
display(stocks[1])

In [ ]:
stocks = oil.weekly_stocks()

In [ ]:
display(stocks)

In [ ]:
net_imports[-1]

# changes compared to last week

In [9]:
stocks = oil.weekly_stocks().sort_index()
stock_change = stocks.diff(1)
stock_change.rename('stock_change', inplace=True)

In [10]:
product_supplied = oil.weekly_product_supplied()
production = oil.weekly_field_production().sort_index()
imp, exp = oil.imports_exports(only_crude=True)
net_imports = imp - exp
net_imports.rename('net_imports', inplace=True)

In [12]:
display(product_supplied.tail())

In [12]:
master_df = pd.merge(production, net_imports, on='period')
master_df = pd.merge(master_df, stock_change, on='period')

In [12]:
master_df['prod_supplied'] = master_df['weekly_field_production'] + master_df['net_imports'] - master_df['stock_change']

In [13]:
display(master_df.tail())

/Users/nickgault/PycharmProjects/Kayak/Oil_analysis/USA_oil_data.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('period').sum()['value']


In [14]:
display(product_supplied.tail())

In [15]:
stock_change[stock_change.index > start_date].plot.bar(figsize=(15,5))